In [26]:
import pandas as pd
import numpy as np
import os
from glob import glob
import sys
import warnings
from time import time
from pandas.tseries.offsets import *

### Santa Monica

In [28]:
cd

/Users/karatechop


In [29]:
cd /Users/karatechop/Library/CloudStorage/GoogleDrive-kc1558628@berkeley.edu/My Drive/ENERES 131 FINAL PROJECT (1)/data/buoy/santa monica 46025

/Users/karatechop/Library/CloudStorage/GoogleDrive-kc1558628@berkeley.edu/My Drive/ENERES 131 FINAL PROJECT (1)/data/buoy/santa monica 46025


In [30]:
def fix_wspd(df): 
    if 'WSPD' in df.columns:
        return 
    else: 
        for i in df.columns:
            if 'WSPD' in i and i != 'WSPD': 
                print(i)
                correct_wspd = int(input("Which index is WSPD at?"))
                df.rename(columns={i:'WSPD'},inplace=True)
                df['WSPD'] = [(i.split()[correct_wspd]) for i in df['WSPD']]

# df_test = pd.read_fwf('ljpc1h2013.txt')
# fix_wspd(df_test)

It appears we're lacking air temperature data for every year. 

In [31]:
# default = ['#YY', 'MM', 'DD', 'hh', 'mm']

# for i in range(2009,2020):
#     try:
#         df_test = pd.read_fwf(f'46225h{i}.txt')
#     #     (df_test.columns[:5] == default).all()
#         print(df_test.loc[1:,'ATMP'].isna().sum(),'\n')
#     except:
#         pass

In [43]:
PATH = os.getcwd()
EXT = "*.txt"
default = ['#YY', 'MM', 'DD', 'hh', 'mm']
count = 0


for path, subdir, files in os.walk(PATH):
    for file in glob(os.path.join(path, EXT)):
        if 'jordan' not in file.lower() and 'all_buoy_years' not in file.lower() and 'copy' not in file.lower() and '.txt' in file.lower():
            if count == 0:
                main = pd.read_fwf(file)
                
                print(file[-14:],'\n',main.columns)
                
                if (main.columns[:5]==default).all():
                    main[['year', 'month', 'day', 'hour', 'minute']] = main[['#YY', 'MM', 'DD', 'hh', 'mm']]
                    if 'ATMP' in main.columns:
                        main['air_temp'] = main['ATMP']
                    else:
                        main['air_temp'] = main[input("Select air_temp col: \n\n")]
                else:
                    main['year'] = main[input("Select year col: ")]
                    main['month'] = main[input("Select month col: ")]
                    main['day'] = main[input("Select day col: ")]
                    main['hour'] = main[input("Select hour col: ")]
                    main['minute'] = main[input("Select minute col: ")]
                    main['air_temp'] = main[input("Select air_temp col: \n\n")]
                
                fix_wspd(main)


                main = main[['year','month','day','hour','minute','WSPD','air_temp']].iloc[1:,:].astype(float)
                count+=1
                
                
                
            else: 
                df = pd.read_fwf(file)
                
                print(file[-14:],'\n',df.columns)
                
                if (df.columns[:5]==default).all():
                    df[['year', 'month', 'day', 'hour', 'minute']] = df[['#YY', 'MM', 'DD', 'hh', 'mm']]
                    if 'ATMP' in df.columns:
                        df['air_temp'] = df['ATMP']
                    else:
                        df['air_temp'] = df[input("Select air_temp col: \n\n")]
                else:
                    df['year'] = df[input("Select year col: ")]
                    df['month'] = df[input("Select month col: ")]
                    df['day'] = df[input("Select day col: ")]
                    df['hour'] = df[input("Select hour col: ")]
                    df['minute'] = df[input("Select minute col: ")]
                    df['air_temp'] = df[input("Select air_temp col: \n\n")]

                fix_wspd(df)


                df = df[['year','month','day','hour','minute','WSPD','air_temp']].iloc[1:,:].astype(float)
                main = pd.concat([main,df]) 
                count+=1

46025h2014.txt 
 Index(['#YY', 'MM', 'DD', 'hh', 'mm', 'WDIR', 'WSPD', 'GST', 'WVHT', 'DPD',
       'APD', 'MWD   PRES', 'ATMP', 'WTMP', 'DEWP', 'VIS', 'TIDE'],
      dtype='object')
46025h2015.txt 
 Index(['#YY', 'MM', 'DD', 'hh', 'mm', 'WDIR', 'WSPD', 'GST', 'WVHT', 'DPD',
       'APD', 'MWD', 'PRES', 'ATMP', 'WTMP', 'DEWP', 'VIS', 'TIDE'],
      dtype='object')
46025h2017.txt 
 Index(['#YY', 'MM', 'DD', 'hh', 'mm', 'WDIR', 'WSPD', 'GST', 'WVHT', 'DPD',
       'APD', 'MWD   PRES', 'ATMP', 'WTMP', 'DEWP', 'VIS', 'TIDE'],
      dtype='object')
46025h2016.txt 
 Index(['#YY', 'MM', 'DD', 'hh', 'mm', 'WDIR WSPD GST', 'WVHT', 'DPD', 'APD',
       'MWD   PRES', 'ATMP', 'WTMP', 'DEWP', 'VIS', 'TIDE'],
      dtype='object')
WDIR WSPD GST
Which index is WSPD at?1
46025h2012.txt 
 Index(['#YY', 'MM', 'DD', 'hh', 'mm', 'WDIR', 'WSPD', 'GST', 'WVHT', 'DPD',
       'APD', 'MWD   PRES', 'ATMP', 'WTMP', 'DEWP', 'VIS', 'TIDE'],
      dtype='object')
46025h2013.txt 
 Index(['#YY', 'MM', 'DD', 'hh', 'm

In [45]:
main = main.replace(999.0,None)
main = main.replace(99.0,None)
main['datetime'] = pd.to_datetime(main[['year','month','day','hour']])
main['datetime'] = main['datetime'] + Week(weekday=1)
main['datetime'] = main['datetime'].dt.strftime('%Y-%m-%d')
main = main[['datetime','WSPD','air_temp']].groupby(['datetime'],as_index=False).mean()
main['buoy'] = '46025 SANTA MONICA'
main.to_csv('all_buoy_years_46025.csv',index=False)
main

,datetime,WSPD,air_temp,buoy
0,2008-01-08,4.554167,13.659524,46025 SANTA MONICA
1,2008-01-15,2.590476,14.248810,46025 SANTA MONICA
2,2008-01-22,3.725000,13.461905,46025 SANTA MONICA
3,2008-01-29,7.082738,12.559524,46025 SANTA MONICA
4,2008-02-05,6.453254,12.268047,46025 SANTA MONICA
...,...,...,...,...
697,2021-12-07,2.385317,13.304467,46025 SANTA MONICA
698,2021-12-14,4.059086,NaN,46025 SANTA MONICA
699,2021-12-21,4.661607,13.359259,46025 SANTA MONICA
700,2021-12-28,4.711310,13.107341,46025 SANTA MONICA


In [50]:
print(main.shape,'\n')
print(main.isna().sum())

(702, 4) 

datetime     0
WSPD         0
air_temp    45
buoy         0
dtype: int64


datetime     0
WSPD         0
air_temp    45
buoy         0
dtype: int64

### 46225 Torrey Pines

In [2]:
cd

/Users/karatechop


In [3]:
cd /Users/karatechop/Library/CloudStorage/GoogleDrive-kc1558628@berkeley.edu/My Drive/ENERES 131 FINAL PROJECT (1)/data/buoy/buoy Station 46225 - Torrey Pines Outer, CA (100)

/Users/karatechop/Library/CloudStorage/GoogleDrive-kc1558628@berkeley.edu/My Drive/ENERES 131 FINAL PROJECT (1)/data/buoy/buoy Station 46225 - Torrey Pines Outer, CA (100)


In [11]:
test['WSPD'].value_counts()

999 99.0 99.0    204505
Name: WSPD, dtype: int64

In [10]:
test = pd.read_csv('all_buoy_years_073_lajolla.csv')
test

,#YY,MM,DD,hh,mm,WSPD,WVHT,DPD,APD,MWD PRES,ATMP,WTMP,DEWP,VIS,TIDE
0,2015,1,1,0,6,999 99.0 99.0,0.88,13.33,5.50,273 9999.0,999.0,16.9,999.0,99.0,99.0
1,2015,1,1,0,36,999 99.0 99.0,0.88,14.29,5.77,260 9999.0,999.0,16.9,999.0,99.0,99.0
2,2015,1,1,1,6,999 99.0 99.0,0.75,14.29,5.63,264 9999.0,999.0,16.9,999.0,99.0,99.0
3,2015,1,1,1,36,999 99.0 99.0,0.72,9.88,6.08,274 9999.0,999.0,16.9,999.0,99.0,99.0
4,2015,1,1,2,6,999 99.0 99.0,0.74,12.50,6.56,267 9999.0,999.0,16.9,999.0,99.0,99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204500,A (100)/46225h2018,12,31,21,30,999 99.0 99.0,1.62,15.38,6.07,273 9999.0,999.0,16.7,999.0,99.0,99.0
204501,A (100)/46225h2018,12,31,22,0,999 99.0 99.0,1.68,9.88,5.99,277 9999.0,999.0,16.7,999.0,99.0,99.0
204502,A (100)/46225h2018,12,31,22,30,999 99.0 99.0,1.69,7.69,6.22,280 9999.0,999.0,16.7,999.0,99.0,99.0
204503,A (100)/46225h2018,12,31,23,0,999 99.0 99.0,1.59,16.67,6.07,260 9999.0,999.0,16.7,999.0,99.0,99.0


In [624]:
print(main.columns)

Index(['year', 'month', 'day', 'hour', 'minute', 'WSPD', 'air_temp',
       'datetime', 'buoy'],
      dtype='object')


In [625]:
def fix_wspd(df): 
    if 'WSPD' in df.columns:
        return 
    else: 
        for i in df.columns:
            if 'WSPD' in i and i != 'WSPD': 
                print(i)
                correct_wspd = int(input("Which index is WSPD at?"))
                df.rename(columns={i:'WSPD'},inplace=True)
                df['WSPD'] = [(i.split()[correct_wspd]) for i in df['WSPD']]

# df_test = pd.read_fwf('ljpc1h2013.txt')
# fix_wspd(df_test)

It appears we're lacking air temperature data for every year. 

In [626]:
default = ['#YY', 'MM', 'DD', 'hh', 'mm']

for i in range(2009,2020):
    try:
        df_test = pd.read_fwf(f'46225h{i}.txt')
    #     (df_test.columns[:5] == default).all()
        print(df_test.loc[1:,'ATMP'].isna().sum(),'\n')
    except:
        pass

0 

0 

0 

0 

0 

0 

0 

0 

0 

0 

0 



In [627]:
PATH = os.getcwd()
EXT = "*.txt"
default = ['#YY', 'MM', 'DD', 'hh', 'mm']
count = 0


for path, subdir, files in os.walk(PATH):
    for file in glob(os.path.join(path, EXT)):
        if 'jordan' not in file.lower() and 'all_buoy_years' not in file.lower() and 'copy' not in file.lower() and '.txt' in file.lower():
            if count == 0:
                main = pd.read_fwf(file)
                
                print(file[-14:],'\n',main.columns)
                
                if (main.columns[:5]==default).all():
                    main[['year', 'month', 'day', 'hour', 'minute']] = main[['#YY', 'MM', 'DD', 'hh', 'mm']]
                    if 'ATMP' in main.columns:
                        main['air_temp'] = main['ATMP']
                    else:
                        main['air_temp'] = main[input("Select air_temp col: \n\n")]
                else:
                    main['year'] = main[input("Select year col: ")]
                    main['month'] = main[input("Select month col: ")]
                    main['day'] = main[input("Select day col: ")]
                    main['hour'] = main[input("Select hour col: ")]
                    main['minute'] = main[input("Select minute col: ")]
                    main['air_temp'] = main[input("Select air_temp col: \n\n")]
                
                fix_wspd(main)


                main = main[['year','month','day','hour','minute','WSPD','air_temp']].iloc[1:,:].astype(float)
                count+=1
                
                
                
            else: 
                df = pd.read_fwf(file)
                
                print(file[-14:],'\n',df.columns)
                
                if (df.columns[:5]==default).all():
                    df[['year', 'month', 'day', 'hour', 'minute']] = df[['#YY', 'MM', 'DD', 'hh', 'mm']]
                    if 'ATMP' in df.columns:
                        df['air_temp'] = df['ATMP']
                    else:
                        df['air_temp'] = df[input("Select air_temp col: \n\n")]
                else:
                    df['year'] = df[input("Select year col: ")]
                    df['month'] = df[input("Select month col: ")]
                    df['day'] = df[input("Select day col: ")]
                    df['hour'] = df[input("Select hour col: ")]
                    df['minute'] = df[input("Select minute col: ")]
                    df['air_temp'] = df[input("Select air_temp col: \n\n")]

                fix_wspd(df)


                df = df[['year','month','day','hour','minute','WSPD','air_temp']].iloc[1:,:].astype(float)
                main = pd.concat([main,df]) 
                count+=1

46225h2015.txt 
 Index(['#YY', 'MM', 'DD', 'hh', 'mm', 'WDIR WSPD GST', 'WVHT', 'DPD', 'APD',
       'MWD   PRES', 'ATMP', 'WTMP', 'DEWP', 'VIS', 'TIDE'],
      dtype='object')
WDIR WSPD GST
Which index is WSPD at?1
46225h2014.txt 
 Index(['#YY', 'MM', 'DD', 'hh', 'mm', 'WDIR WSPD GST', 'WVHT', 'DPD', 'APD',
       'MWD   PRES', 'ATMP', 'WTMP', 'DEWP', 'VIS', 'TIDE'],
      dtype='object')
WDIR WSPD GST
Which index is WSPD at?1
46225h2016.txt 
 Index(['#YY', 'MM', 'DD', 'hh', 'mm', 'WDIR WSPD GST', 'WVHT', 'DPD', 'APD',
       'MWD   PRES', 'ATMP', 'WTMP', 'DEWP', 'VIS', 'TIDE'],
      dtype='object')
WDIR WSPD GST
Which index is WSPD at?1
46225h2017.txt 
 Index(['#YY', 'MM', 'DD', 'hh', 'mm', 'WDIR WSPD GST', 'WVHT', 'DPD', 'APD',
       'MWD   PRES', 'ATMP', 'WTMP', 'DEWP', 'VIS', 'TIDE'],
      dtype='object')
WDIR WSPD GST
Which index is WSPD at?1
46225h2013.txt 
 Index(['#YY', 'MM', 'DD', 'hh', 'mm', 'WDIR WSPD GST', 'WVHT', 'DPD', 'APD',
       'MWD   PRES', 'ATMP', 'WTMP', 'DEWP

In [628]:
main = main.replace(999.0,None)
main = main.replace(99.0,None)
main['datetime'] = pd.to_datetime(main[['year','month','day','hour']])
main['buoy'] = '46225 OUTER'
main
main.to_csv('all_buoy_years_46225.csv',index=False)

In [629]:
main.isna().sum()

year             0
month            0
day              0
hour             0
minute           0
WSPD        238373
air_temp    238373
datetime         0
buoy             0
dtype: int64

### 073 La Jolla

In [605]:
cd

/Users/karatechop


In [606]:
cd /Users/karatechop/Library/CloudStorage/GoogleDrive-kc1558628@berkeley.edu/My Drive/ENERES 131 FINAL PROJECT (1)/data/buoy/buoy Station LJPC1 - La Jolla, CA (073)/

/Users/karatechop/Library/CloudStorage/GoogleDrive-kc1558628@berkeley.edu/My Drive/ENERES 131 FINAL PROJECT (1)/data/buoy/buoy Station LJPC1 - La Jolla, CA (073)


In [607]:
ls

all_buoy_years_073.csv          ljpc1h2013.txt
all_buoy_years_073_lajolla.csv  ljpc1h2013.txt.gz
all_buoy_years_46225.csv        ljpc1h2014.txt
ljpc1h2005.txt                  ljpc1h2014.txt.gz
ljpc1h2005.txt.gz               ljpc1h2015.txt
ljpc1h2006.txt                  ljpc1h2015.txt.gz
ljpc1h2006.txt.gz               ljpc1h2016.txt
ljpc1h2007.txt                  ljpc1h2016.txt.gz
ljpc1h2007.txt.gz               ljpc1h2017.txt
ljpc1h2008.txt                  ljpc1h2017.txt.gz
ljpc1h2008.txt.gz               ljpc1h2018.txt
ljpc1h2009.txt                  ljpc1h2018.txt.gz
ljpc1h2009.txt.gz               ljpc1h2019.txt
ljpc1h2010.txt                  ljpc1h2019.txt.gz
ljpc1h2010.txt.gz               ljpc1h2020.txt
ljpc1h2011.txt                  ljpc1h2020.txt.gz
ljpc1h2011.txt.gz               ljpc1h2021.txt
ljpc1h2012.txt                  ljpc1h2021.txt.gz
ljpc1h2012.txt.gz


In [609]:
PATH = os.getcwd()
EXT = "*.txt"
default = ['#YY', 'MM', 'DD', 'hh', 'mm']
count = 0


for path, subdir, files in os.walk(PATH):
    for file in glob(os.path.join(path, EXT)):
        if 'jordan' not in file.lower() and 'all_buoy_years' not in file.lower() and 'copy' not in file.lower() and '.txt' in file.lower():
            if count == 0:
                main = pd.read_fwf(file)
                
                print(file[-14:],'\n',main.columns)
                
                if (main.columns[:5]==default).all():
                    main[['year', 'month', 'day', 'hour', 'minute']] = main[['#YY', 'MM', 'DD', 'hh', 'mm']]
                    if 'ATMP' in main.columns:
                        main['air_temp'] = main['ATMP']
                    else:
                        main['air_temp'] = main[input("Select air_temp col: \n\n")]
                else:
                    main['year'] = main[input("Select year col: ")]
                    main['month'] = main[input("Select month col: ")]
                    main['day'] = main[input("Select day col: ")]
                    main['hour'] = main[input("Select hour col: ")]
                    main['minute'] = main[input("Select minute col: ")]
                    main['air_temp'] = main[input("Select air_temp col: \n\n")]
                
                fix_wspd(main)


                main = main[['year','month','day','hour','minute','WSPD','air_temp']].iloc[1:,:].astype(float)
                count+=1
                
                
                
            else: 
                df = pd.read_fwf(file)
                
                print(file[-14:],'\n',df.columns)
                
                if (df.columns[:5]==default).all():
                    df[['year', 'month', 'day', 'hour', 'minute']] = df[['#YY', 'MM', 'DD', 'hh', 'mm']]
                    if 'ATMP' in df.columns:
                        df['air_temp'] = df['ATMP']
                    else:
                        df['air_temp'] = df[input("Select air_temp col: \n\n")]
                else:
                    df['year'] = df[input("Select year col: ")]
                    df['month'] = df[input("Select month col: ")]
                    df['day'] = df[input("Select day col: ")]
                    df['hour'] = df[input("Select hour col: ")]
                    df['minute'] = df[input("Select minute col: ")]
                    df['air_temp'] = df[input("Select air_temp col: \n\n")]

                fix_wspd(df)


                df = df[['year','month','day','hour','minute','WSPD','air_temp']].iloc[1:,:].astype(float)
                main = pd.concat([main,df]) 
                count+=1

ljpc1h2018.txt 
 Index(['#YY', 'MM', 'DD', 'hh', 'mm', 'WDIR', 'WSPD GST', 'WVHT', 'DPD', 'APD',
       'MWD   PRES', 'ATMP', 'WTMP', 'DEWP', 'VIS', 'TIDE'],
      dtype='object')
WSPD GST
Which index is WSPD at?0
ljpc1h2019.txt 
 Index(['#YY', 'MM', 'DD', 'hh', 'mm', 'WDIR', 'WSPD GST', 'WVHT', 'DPD', 'APD',
       'MWD   PRES', 'ATMP', 'WTMP', 'DEWP', 'VIS', 'TIDE'],
      dtype='object')
WSPD GST
Which index is WSPD at?0
ljpc1h2009.txt 
 Index(['#YY', 'MM', 'DD', 'hh', 'mm', 'WDIR', 'WSPD', 'GST', 'WVHT', 'DPD',
       'APD', 'MWD   PRES', 'ATMP', 'WTMP', 'DEWP', 'VIS', 'TIDE'],
      dtype='object')
ljpc1h2021.txt 
 Index(['#YY', 'MM', 'DD', 'hh', 'mm', 'WDIR', 'WSPD', 'GST', 'WVHT', 'DPD',
       'APD', 'MWD   PRES', 'ATMP', 'WTMP', 'DEWP', 'VIS', 'TIDE'],
      dtype='object')
ljpc1h2020.txt 
 Index(['#YY', 'MM', 'DD', 'hh', 'mm', 'WDIR', 'WSPD', 'GST', 'WVHT', 'DPD',
       'APD', 'MWD   PRES', 'ATMP', 'WTMP', 'DEWP', 'VIS', 'TIDE'],
      dtype='object')
ljpc1h2008.txt 
 Index(

In [610]:
main = main.replace(999.0,None)
main = main.replace(99.0,None)
main['datetime'] = pd.to_datetime(main[['year','month','day','hour']])
main['buoy'] = '073 INNER'
main
main.to_csv('all_buoy_years_073.csv',index=False)

In [611]:
main.isna().sum()

year             0
month            0
day              0
hour             0
minute           0
WSPD         23701
air_temp    102325
datetime         0
buoy             0
dtype: int64

In [497]:
main['air_temp'].isna().sum()/main['air_temp'].shape[0]

0.7905207045735476

## Joining the Two Buoys

In [662]:
from pandas.tseries.offsets import *

In [660]:
torey_46225 = pd.read_csv('all_buoy_years_46225.csv')
torey_46225['datetime'] = pd.to_datetime(torey_46225['datetime'])
torey_46225 = torey_46225.rename(columns={'WSPD':'WSPD_46225',
                                          'air_temp':'air_temp_46225'})[['datetime','WSPD_46225','air_temp_46225']]

torey_46225['datetime'] = torey_46225['datetime'] + Week(weekday=1)
torey_46225['datetime'] = torey_46225['datetime'].dt.strftime('%Y-%m-%d')
torey_46225 = torey_46225.groupby(['datetime'],as_index=False).mean()

print(torey_46225.shape,torey_46225.isna().sum())
torey_46225

(727, 3) datetime            0
WSPD_46225        727
air_temp_46225    727
dtype: int64


,datetime,WSPD_46225,air_temp_46225
0,2008-01-08,NaN,NaN
1,2008-01-15,NaN,NaN
2,2008-01-22,NaN,NaN
3,2008-01-29,NaN,NaN
4,2008-02-05,NaN,NaN
...,...,...,...
722,2021-12-07,NaN,NaN
723,2021-12-14,NaN,NaN
724,2021-12-21,NaN,NaN
725,2021-12-28,NaN,NaN


In [654]:
cd /Users/karatechop/Library/CloudStorage/GoogleDrive-kc1558628@berkeley.edu/My Drive/ENERES 131 FINAL PROJECT (1)/data/buoy/buoy Station LJPC1 - La Jolla, CA (073)/

/Users/karatechop/Library/CloudStorage/GoogleDrive-kc1558628@berkeley.edu/My Drive/ENERES 131 FINAL PROJECT (1)/data/buoy/buoy Station LJPC1 - La Jolla, CA (073)


In [656]:
la_jolla_073 = pd.read_csv('all_buoy_years_073.csv')
la_jolla_073['datetime'] = pd.to_datetime(la_jolla_073['datetime'])
la_jolla_073 = la_jolla_073.rename(columns={'WSPD':'WSPD_073',
                                          'air_temp':'air_temp_073'})[['datetime','WSPD_073','air_temp_073']]

la_jolla_073['datetime'] = la_jolla_073['datetime'] + Week(weekday=1)
la_jolla_073['datetime'] = la_jolla_073['datetime'].dt.strftime('%Y-%m-%d')
la_jolla_073 = la_jolla_073.groupby(['datetime'],as_index=False).mean()

print(la_jolla_073.shape,la_jolla_073.isna().sum())
la_jolla_073

(811, 3) datetime          0
WSPD_073        152
air_temp_073    636
dtype: int64


,datetime,WSPD_073,air_temp_073
0,2005-02-01,3.028571,12.287143
1,2005-02-08,3.357616,12.983444
2,2005-02-15,1.668712,14.219632
3,2005-02-22,1.421053,15.932558
4,2005-03-01,1.746914,14.328571
...,...,...,...
806,2022-01-04,1.923494,NaN
807,2022-01-11,2.817857,NaN
808,2022-01-18,2.691018,NaN
809,2022-01-25,2.910119,NaN


In [663]:
final = pd.merge(la_jolla_073,torey_46225,on='datetime',how='outer')
# final['datetime'] = pd.to_datetime(final['datetime'].dt.strftime('%Y-%M'))
final.to_csv('lj_torey_buoys.csv',index=False)

In [664]:
final

,datetime,WSPD_073,air_temp_073,WSPD_46225,air_temp_46225
0,2005-02-01,3.028571,12.287143,NaN,NaN
1,2005-02-08,3.357616,12.983444,NaN,NaN
2,2005-02-15,1.668712,14.219632,NaN,NaN
3,2005-02-22,1.421053,15.932558,NaN,NaN
4,2005-03-01,1.746914,14.328571,NaN,NaN
...,...,...,...,...,...
877,2010-12-21,NaN,NaN,NaN,NaN
878,2012-10-16,NaN,NaN,NaN,NaN
879,2013-02-19,NaN,NaN,NaN,NaN
880,2013-02-26,NaN,NaN,NaN,NaN
